In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pyspark-test-data/test1.csv
/kaggle/input/pyspark-test-data/test2.csv
/kaggle/input/pyspark-test-data/tips.csv
/kaggle/input/pyspark-test-data/test3.csv


In [2]:
!pip install pyspark
import pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 3.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=6b50ba6c6d8028c4d666bfe7669cd41b74a8d0577d4aa3cd65889c08eaffd1e4
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/20 18:14:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
## Read The dataset
training = spark.read.csv('../input/pyspark-test-data/test1.csv',header=True,inferSchema=True)

In [5]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [8]:
########################################################################################
# In pandas what we do is 1st divide data to indpendent features and dependent features
# We use X and Y variable
# Then we do train/ test split
########################################################################################

# In PySpark we do different techniques
# We have to basically create a way where I can group all my independent features
# [Age,Experience]----> new feature--->independent feature
# to do the grouping we use vector assembler

from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [9]:
output=featureassembler.transform(training)

In [10]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [11]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [12]:
finalized_data=output.select("Independent Features","Salary")

In [13]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [14]:
from pyspark.ml.regression import LinearRegression

##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

22/07/20 18:14:34 WARN Instrumentation: [82296a21] regParam is zero, which might cause numerical instability and overfitting.


In [15]:
### Coefficients
regressor.coefficients

DenseVector([-204.0816, 1632.6531])

In [16]:
### Intercepts
regressor.intercept

20000.0

In [17]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [18]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|17346.938775510207|
|          [23.0,2.0]| 18000|18571.428571428572|
|          [29.0,4.0]| 20000|20612.244897959186|
|          [30.0,8.0]| 25000|26938.775510204083|
+--------------------+------+------------------+



In [19]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1367.3469387755122, 2492086.6305706007)